In [1]:
using Pkg: @pkg_str

pkg"activate .."
pkg"status"

Project Arborist v0.1.0
    Status `~/JuliaEnvs/Arborist/Project.toml`
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1
    Status `~/JuliaEnvs/Arborist/Manifest.toml`
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1


In [2]:
using CodeTracking
using MacroTools

In [3]:
includet("../test/demo.jl")

In [16]:
get_ast(f, args...) = get_ast(@which f(args...))
function get_ast(::Type{F}, args...) where F <: Function
    return get_ast(F.instance, args...)
end
function get_ast(meth::Method)
    ast = definition(meth)
    return MacroTools.striplines(ast)
end

get_ast (generic function with 3 methods)

In [41]:
get_ast(typeof(bar), 1)

:(bar(x) = begin
          x
      end)

In [76]:
@generated function graft(f, call_args...)
    ast = get_ast(f, call_args...)
    
    # TODO: https://github.com/MikeInnes/MacroTools.jl/issues/116
    def = MacroTools.splitdef(ast)
    
    body = Expr(:block,
        Expr(:(=),  # give args their local names
            Expr(:tuple, def[:args]...),
            :call_args,
        ),
        def[:body]
    )
    
    MacroTools.postwalk(body) do expr
        if isexpr(expr, :call)
            # TODO: make this ref `graft` right when in module
            # TODO: make this do kwargs right
            Expr(:block,
                :(println("Calling ", $(expr.args[1]))),
                Expr(:call, :graft, expr.args...)
            )
        else
            expr
        end
    end
end

graft (generic function with 1 method)

In [79]:
graft(foo)

Calling bar


2

In [58]:
true_args = Any[:x, :y]
:($true_args = [1,2])

:(Any[:x, :y] = [1, 2])

In [63]:
(x,y) = [1,2]
x, y

(1, 2)

In [64]:
:((x, y) = [1,2]) |> dump

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Expr
      head: Symbol tuple
      args: Array{Any}((2,))
        1: Symbol x
        2: Symbol y
    2: Expr
      head: Symbol vect
      args: Array{Any}((2,))
        1: Int64 1
        2: Int64 2


In [75]:
@generated function testem1(args...)
    true_args = Any[:x, :y]

end

testem1(2,37) |> dump

Int64 39


In [30]:
quote
    println("Calling ", 1)
    foo()
    end |> dump

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[30]
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol println
        2: String "Calling "
        3: Int64 1
    3: LineNumberNode
      line: Int64 3
      file: Symbol In[30]
    4: Expr
      head: Symbol call
      args: Array{Any}((1,))
        1: Symbol foo
